In [1]:
import dask
import logging
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=10)

from grizly.scheduling.registry import RedisDB, Trigger, Job

## RedisDB

In [2]:
rdb = RedisDB()

2020-08-20 08:58:54,006 - grizly.config - DEBUG - Config data loaded from /home/analyst/.grizly/config.json.


In [3]:
rdb.get_jobs()

[]

In [4]:
rdb.get_triggers()

[]

In [5]:
rdb.get_job_runs()

[]

## Job

In [6]:
@dask.delayed
def print_time():
    print(f"Hello, it's {datetime.now().__str__()}")
    
time_log = print_time()

@dask.delayed
def add(x, y):
    return x + y

_sum = add(1, 2)

In [7]:
job1 = Job("test_print")
job1.register(tasks=[time_log], crons="* * * * *", if_exists="replace")

job2 = Job("test_sum")
job2.register(tasks=[_sum], upstream="test_print", if_exists="replace")

2020-08-20 08:58:54,272 - grizly.scheduling.registry - DEBUG - Job(name='test_print') with cron '* * * * *' has been added to rq sheduler with id ad8e61cf-9cd5-481f-ab9b-1501db997e02
2020-08-20 08:58:54,280 - grizly.scheduling.registry - INFO - Job(name='test_print') has been added to the scheduler
2020-08-20 08:58:54,297 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully registered
2020-08-20 08:58:54,322 - grizly.scheduling.registry - INFO - Adding ['test_sum'] to Job(name='test_print').downstream...
2020-08-20 08:58:54,335 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully registered


Job(name='test_sum')

In [8]:
job1.info()

name: test_print
owner: None
created_at: 2020-08-20 08:58:54.249979+00:00
crons: ['* * * * *']
downstream: [Job(name='test_sum')]
upstream: []
triggers: []


In [9]:
job1.runs

[]

In [10]:
job2.info()

name: test_sum
owner: None
created_at: 2020-08-20 08:58:54.306279+00:00
crons: []
downstream: []
upstream: [Job(name='test_print')]
triggers: []


In [11]:
job2.runs

[]

In [12]:
job1.submit(to_dask=False)

2020-08-20 08:58:54,657 - grizly.scheduling.registry - INFO - Submitting Job(name='test_print')...
2020-08-20 08:58:54,715 - grizly.scheduling.registry - INFO - Job(name='test_print') finished with status success
2020-08-20 08:58:54,717 - grizly.scheduling.registry - INFO - Enqueueing Job(name='test_print').downstream...
2020-08-20 08:58:54,734 - grizly.scheduling.registry - INFO - Job(name='test_sum') has been enqueued


Hello, it's 2020-08-20 08:58:54.711336


[None]

In [13]:
job1.last_run.info()

id: 1
name: None
created_at: 2020-08-20 08:58:54.668836+00:00
finished_at: 2020-08-20 08:58:54.736632+00:00
duration: 0
status: success
error: None


In [14]:
job1.unregister(remove_job_runs=True)

2020-08-20 08:58:54,938 - grizly.scheduling.registry - DEBUG - Rq job ad8e61cf-9cd5-481f-ab9b-1501db997e02 removed from the scheduler
2020-08-20 08:58:54,942 - grizly.scheduling.registry - INFO - Job(name='test_print') has been removed from the scheduler
2020-08-20 08:58:54,999 - grizly.scheduling.registry - INFO - Removing ['test_print'] from Job(name='test_sum').upstream...
2020-08-20 08:58:55,022 - grizly.scheduling.registry - INFO - Removing ['test_sum'] from Job(name='test_print').downstream...
2020-08-20 08:58:55,051 - grizly.scheduling.registry - INFO - Job(name='test_print') successfully removed from registry


In [15]:
job2.unregister()

2020-08-20 08:58:55,124 - grizly.scheduling.registry - INFO - Job(name='test_sum') successfully removed from registry


## ADMIN

#### Unregister all jobs

#### Remove all keys from Redis

#### Cancel all rq scheduler jobs